In [13]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, \
precision_recall_curve, auc

In [14]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [15]:
from fundamentas.stepbystep_v0 import StepByStep 

In [16]:
#data generation 
X , y = make_moons(n_samples= 100 , noise = 0.3 , random_state= 0)


In [18]:
#train test split
X_train , X_val , y_train , y_val = train_test_split(X , y ,test_size=0.2 , random_state= 13)
X_train.shape

(80, 2)

In [20]:
#standaerrize the feature using scikit learn standard scale 
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_val = sc.transform(X_val)

In [21]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [28]:
X_train.shape

(80, 2)

## Data PreParation 

In [45]:
torch.manual_seed(42)
X_train_tensor = torch.as_tensor(X_train).float()
y_train_tensor = torch.as_tensor(y_train).view(-1,1).float()

X_val_tensor = torch.as_tensor(X_val).float()
y_val_tensor = torch.as_tensor(y_val).view(-1 , 1).float()

#make data set 
train_dataset = TensorDataset(X_train_tensor , y_train_tensor)
test_dataset = TensorDataset(X_val_tensor , y_val_tensor)

#build data loader 
train_loader = DataLoader(
    dataset= train_dataset , 
    shuffle= True , 
    batch_size= 16 )

val_loader = DataLoader(
    dataset= test_dataset , 
    batch_size = 16
)


There are two countries  , Country A  , Countrty B 
Country A has 75 percent of wining 
Country B has 100 - 75 -> 25% wining change

Wining odds are [probability]
3 to 1 (75 / 25)
and 1 to 3 (25 / 75) 

In [58]:
def ods_ratio(prob):
    return prob/(1-prob)

p = .75 
q = 1-p
ods_ratio(p) , ods_ratio(q) 
#note that these are the probabilities

(3.0, 0.3333333333333333)

# map probability in real number 

In [59]:
#by taking log of odds ratio , the funciton make its symemetrical and also maps probabilities into real numbers 
def log_odds_ratio(prob):
    return np.log(ods_ratio(prob))

p = .75
q = 1 - p 

log_odds_ratio(p) , log_odds_ratio(q)


(1.0986122886681098, -1.0986122886681098)

In [65]:
#sigmoid 
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
p = .75
q = 1-p
sigmoid(log_odds_ratio(p)) , sigmoid(log_odds_ratio(q))

(0.75, 0.25)

In [68]:
#sequential model to build own logistic regression in pytorch
torch.manual_seed(42)
model1 = nn.Sequential()
model1.add_module('linear' , nn.Linear(2 ,1))
model1.add_module('sigmoid' , nn.Sigmoid()) 
model1.state_dict()

OrderedDict([('linear.weight', tensor([[0.5406, 0.5869]])),
             ('linear.bias', tensor([-0.1657]))])

## Loss 

#### let assume two dummy data points  
#### assume our model make prediction for them 0.9 and 0.2 
##### i.e 90 percent probability for being positive and 20% of being positive for actual negative 

In [90]:
dummy_lables = torch.tensor([1.0 , 0.0])
dummy_predictions = torch.tensor([0.9 , .2])

# positve class (lables = 1 )
positive_pred = dummy_predictions[dummy_lables== 1 ]
first_sumation = torch.log(positive_pred).sum()

negative_pred = dummy_predictions[dummy_lables == 0]
second_sumation = torch.log( 1 - negative_pred).sum()

# n total 
n_total = dummy_lables.size(0)
loss  = -(first_sumation + second_sumation) / n_total
loss 


tensor(0.1643)

first sum add's error corresponding to positive class ,
 second sums add's erros corresponing to negative class 

# BCELOSS

nn.BCELoss() is a higher function takes two argument 
1) reduction mean , sum, none 
2) weight , default is none 

In [91]:
loss_fn = nn.BCELoss(reduce= 'mean')
loss_fn

d:\software\miniconda\envs\mlenv\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


BCELoss()

## important  , always pass prediction first and then laels  , the order matters 

In [92]:
rightway_loss = loss_fn(dummy_predictions , dummy_lables)
wrongway_loss = loss_fn(dummy_lables , dummy_predictions)
rightway_loss , wrongway_loss 

(tensor(0.1643), tensor(15.0000))

#### Another Binary Cross Entropy
### BCEWITHLOGITS 
### nn.BECWITHLogitsLoss()

In [ ]:
nn.BEc